In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import OrderedDict
import math

In [2]:
#Hadamard matrices

H = {}
H[4] = torch.tensor([[1.0, 1.0, 1.0, 1.0], [1.0, -1.0, 1.0, -1.0], [1.0, 1.0, -1.0, -1.0], [1.0, -1.0, -1.0, 1.0]])
H[8] = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
                            [1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0],
                            [1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0],
                            [1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0],
                            [1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0],
                            [1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0],
                            [1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0],
                            [1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0]])


f = open("had64.txt")
H64 = []
for i in range(64):
    H64.append([1.0 if i=='+' else -1.0 for i in list(f.readline())[:-1]])
    
H[64] = H64

In [161]:
#Generating boolean functions and from them, their walsh spectra to feed as inputs to the neural network

#k = number of variables in the boolean functions, n = number of inputs to train the model on
def generate_input_data(k, n, scheme='regular'):
    two_pow_k, data = pow(2, k), []
    if(scheme=='regular'):
        if(n<two_pow_k):
            print("Requested number of inputs less than 2^k. Please request a higher number")
            return data
        while(True):
            if(two_pow_k<=16):
                perm = torch.randperm(pow(2, two_pow_k))
                data = [[0.0]*(two_pow_k-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in perm[:n]]
            else:
                data = [[0.0 if random.random()>0.5 else 1.0 for i in range(two_pow_k)] for i in range(n)]
            rank = np.linalg.matrix_rank(data)
            if(rank<two_pow_k):
                print("Rank (", rank, ") not large enough, generating data again")
            else:
                data = torch.matmul(torch.tensor(data), H[two_pow_k])
                print("Data generated.")
                break
    elif(scheme=='one-hot'):
        data = [[0.0]*i + [1.0] + [0.0]*(two_pow_k-i-1) for i in range(two_pow_k)]
        data = torch.matmul(torch.tensor(data), H[two_pow_k])
        print("Data generated")
    return data

k = 3
two_pow_k = pow(2, k)
n = pow(2, two_pow_k)
walsh_spectra = generate_input_data(k, n)

Data generated.


In [163]:
#Calculating the correlation immunity of functions given their walsh spectra

def correlation_immunity(walsh_spectra):
    ci = []
    n, two_pow_k = walsh_spectra.size()[0], walsh_spectra.size()[1]
    k = int(math.log2(two_pow_k))
    no_ones = []
    for i in range(two_pow_k):
        no_ones.append(sum([int(dig) for dig in bin(i)[2:]]))
    for spectrum in walsh_spectra:
        m_ci = [1]*(k+1)
        for i in range(two_pow_k):
            if(spectrum[i]!=0):
                m_ci[no_ones[i]] = 0
        m = 1
        while(m<k+1 and m_ci[m]==1):
            m+=1
        m -= 1
        # Let ci item = [x0, x1, ..., xm]. x0 is 0 if it is balanced, xi is 1 if it is i-correlation-immune
        ci.append(([1.0] if spectrum[0]==two_pow_k//2 else [0.0])+(m)*[1.0]+(k-m)*[0.0])
    return ci

ci = torch.tensor(correlation_immunity(walsh_spectra))
print("Correlation Immunity calculated")

Correlation Immunity calculated


In [185]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        two_pow_k = pow(2,k)
        self.lin = nn.Linear(two_pow_k, k+1)
#         self.lin.weight.data = torch.tensor([[1.0, 0, 0, 0, 0, 0, 0, 0],
#                                         [0, 1, 1, 0, 1, 0, 0, 0],
#                                         [0, 1, 1, 1, 1, 1, 1, 0],
#                                         [0, 1, 1, 1, 1, 1, 1, 1]])
#         self.lin.bias.data = torch.tensor([0.0, -2, -5, -6])
    
    
    def forward(self, x):
        x = self.lin(x)
        immunity = nn.Hardtanh(0, 1)(x)
        return immunity

# model =  NeuralNetwork()
# weight_dict = OrderedDict(model.named_parameters())
# print(weight_dict)
# print(walsh_spectra[:15])
# inp = torch.tensor([[1.0 if spec[0]==two_pow_k//2 else 0.0] + [1.0 if val==0 else 0.0 for val in spec[1:]] for spec in walsh_spectra[:15]])
# print(inp)
# out = model(inp)
# for i in range(15):
#     print(out[i], ci[i])

In [210]:
class TrainModel():

    def __init__(self, model, device, learningRate, inputDim, epochs):

        self.device = device
        self.net = model.to(self.device)
        self.optimizer = optim.Adam(self.net.parameters(), lr=learningRate)
        self.inputDim = inputDim
        self.epochs = epochs

    def train(self,):

        for epoch in range(self.epochs):

            self.optimizer.zero_grad()
            input = torch.tensor([[1.0 if spec[0]==two_pow_k//2 else 0.0] + [1.0 if val==0 else 0.0 for val in spec[1:]] for spec in walsh_spectra]).to(self.device)
            output = self.net(input)
            loss = F.mse_loss(ci, output)
            loss.backward()
            self.optimizer.step()      

    def test(self,n):
        
        test_samples = torch.tensor([[1.0 if spec[0]==two_pow_k//2 else 0.0] + [1.0 if val==0 else 0.0 for val in spec[1:]] for spec in walsh_spectra[-n:]]).to(self.device)
        preds = torch.round(self.net(test_samples))
        loss = F.mse_loss(ci[-n:], preds)
        print("loss = ", loss)
        print(ci[-n:])
        print(preds)


In [211]:
learningRate = 0.01
epochs = 2000
two_pow_k = pow(2, k)

# Additional Initializer
model =  NeuralNetwork()
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")
trainer = TrainModel(model, device, learningRate, two_pow_k, epochs)
trainer.train()
trainer.test(40)


Using cpu device


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [205]:
weight_dict = OrderedDict(model.named_parameters())
print(weight_dict['lin.weight'])
print(weight_dict['lin.bias'])

Parameter containing:
tensor([[ 1.0045e+00, -1.2289e-04, -3.6357e-04, -4.3695e-04, -4.5267e-04,
         -4.3102e-04, -1.9332e-04, -1.3713e-04],
        [ 9.7369e-08,  1.0000e+00,  1.0000e+00, -3.2380e-07,  1.0000e+00,
          3.3894e-07, -3.3110e-07,  4.8111e-08],
        [-2.5853e-01, -3.3823e-01, -2.5508e-01, -1.4343e-01, -1.9189e-01,
          7.7660e-02,  7.2500e-02,  4.1680e-02],
        [-2.4846e-01, -1.1852e-01,  1.2863e-01, -2.3546e-01, -8.2427e-02,
         -2.2458e-02,  3.3191e-02,  1.1918e-01]], requires_grad=True)
Parameter containing:
tensor([-1.8670e-03, -2.0000e+00, -1.5452e-25, -4.7487e-02],
       requires_grad=True)


In [109]:
weight_dict = OrderedDict(model.named_parameters())
print(weight_dict)
print(walsh_spectra[1])
print(torch.tensor([[1 if val==0 else 0 for val in spec] for spec in walsh_spectra])[1])
# x = torch.tensor([[ 3.3839e-02, -1.1122e-01, -4.7419e-01],
#         [-8.8419e-03, -2.9123e-01, -3.9918e-01],
#         [ 4.7317e-02, -2.3600e-01, -3.7276e-01],
#         [ 3.7765e-01, -2.1951e-02, -6.0302e-01],
#         [ 1.2820e-01,  6.2765e-02, -5.1973e-01]])

# # for i, inp in enumerate(x):
# for inp in x:
#     for imm in inp:
#         if(imm<=0):
#             print(imm)
# # immunity = torch.tensor([torch.tensor([1.0 if imm<=0 else 0.0 for imm in enumerate(inp)]) for inp in enumerate(x)])
# immunity = torch.tensor([[1.0 if imm<=0 else 0.0 for imm in inp] for inp in x])
# print(immunity)

# for x in immunity:
#     print(x)


# print(len([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]))
# print(walsh_spectra[240])
# print(correlation_immunity(walsh_spectra)[240])

OrderedDict([('linear_stack.0.weight', Parameter containing:
tensor([[-0.1614,  0.0411, -0.1521, -0.2886,  0.2480, -0.2408,  0.0709,  0.0664],
        [ 0.0220,  0.0802, -0.0709,  0.1610,  0.0594,  0.0637, -0.0524, -0.0143],
        [-0.1557, -0.0862,  0.1195, -0.0795,  0.0290,  0.1450, -0.1094, -0.0919],
        [-0.3567,  0.1902, -0.1556, -0.1234,  0.1328, -0.1280, -0.2085, -0.1013]],
       requires_grad=True)), ('linear_stack.0.bias', Parameter containing:
tensor([-0.2543, -0.2404, -0.1604, -0.2468], requires_grad=True))])
tensor([ 4.,  0., -2.,  2., -2., -2.,  0.,  0.])
tensor([0, 1, 0, 0, 0, 0, 1, 1])


In [119]:
# m = nn.Hardtanh(0, 1)
# input = torch.tensor([-0.1, 0, 0.5, 1, 2, 3])
# output = m(input)
# print(output)

no_ones = []
for i in range(two_pow_k):
    no_ones.append(sum([int(dig) for dig in bin(i)[2:]]))
print(no_ones)

[0, 1, 1, 2, 1, 2, 2, 3]
